In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install -U googlemaps

Mounted at /content/drive
  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40715 sha256=2b27083086a8152d4bb41a7ad111ccbb9036e1535713d36c2d414808a0ec1c25
  Stored in directory: /root/.cache/pip/wheels/f1/09/77/3cc2f5659cbc62341b30f806aca2b25e6a26c351daa5b1f49a
Successfully built googlemaps


In [ ]:
import json
import os
import pandas as pd
import googlemaps
from datetime import datetime

In [ ]:
#Read Json File and loading data into an array
with open('/content/builker.scrapy_bogota_apartmentsV1.3.0_october_1_2023.json') as f:
  data_json = json.load(f)



In [ ]:
#Print the data structure selecting printing one element into the arrray
data_json[0]

{'_id': {'$oid': '64903166384bbcf0c9bbd349'},
 'codigo': '374-M2718950',
 'tipo_propiedad': 'APARTAMENTO',
 'tipo_operacion': 'VENTA Y ARRIENDO',
 'precio_venta': {'$numberLong': '2500000000'},
 'precio_arriendo': 13500000,
 'area': 320,
 'habitaciones': 4,
 'banos': 5,
 'administracion': 2500000,
 'parqueaderos': 3,
 'sector': 'COUNTRY',
 'estrato': 6,
 'antiguedad': 'MAS DE 20 ANOS',
 'estado': 'USADO',
 'longitud': -74.026855,
 'latitud': 4.7154746,
 'featured_interior': ['ALARMA',
  'CHIMENEA',
  'CITOFONOS',
  'COMEDOR AUXILIAR',
  'TIPO DE CORTINAS PERSIANAS',
  'CUARTO DE SERVICIO',
  'ESTUDIO O BIBLIOTECA',
  'PARQUEADERO CUBIERTO',
  'HALL DE ALCOBAS',
  'JACUZZI',
  'PUERTA SEGURIDAD',
  'SAUNA / TURCO',
  'ZONA DE LAVANDERIA',
  'INSTALACION DE GAS NATURAL',
  'CLOSETS 4',
  'PISO 8',
  'SE PERMITE FUMAR',
  'PERMITE MASCOTAS',
  'CALENTADOR GAS',
  'TIPO DE COCINA OTRA',
  'TIPO DE COMEDOR COMEDOR INDEPENDIENTE',
  'DEPOSITO 1',
  'TIPO DE ESTUFA MIXTA',
  'TIPO DE PARQUEAD

In [ ]:
# Getting the keys of the dict element into the data
data_json[0].keys()

dict_keys(['_id', 'codigo', 'tipo_propiedad', 'tipo_operacion', 'precio_venta', 'precio_arriendo', 'area', 'habitaciones', 'banos', 'administracion', 'parqueaderos', 'sector', 'estrato', 'antiguedad', 'estado', 'longitud', 'latitud', 'featured_interior', 'featured_exterior', 'featured_zona_comun', 'featured_sector', 'descripcion', 'datetime', 'website', 'compañia', 'imagenes', 'last_view'])

In [ ]:
# creating an array with the keys of interest
keys_of_interest = ['codigo', 'tipo_propiedad', 'tipo_operacion',
                    'precio_venta', 'area', 'habitaciones',
                    'banos', 'administracion', 'parqueaderos', 'sector',
                    'estrato', 'antiguedad', 'estado', 'longitud', 'latitud',
                    'datetime',
                    ]

In [ ]:
type(data_json[0]['precio_venta'])
data_json[0]['precio_venta'].keys()
type(data_json[0]['precio_venta'].keys())

dict_keys

In [ ]:
# Defining a Function to parse the element info
def get_instance(element:dict) -> dict:
  instance_element = {}
  for key in keys_of_interest:
    if key in element.keys():
      if key == 'precio_venta' and type(element[key]) == dict:
        sub_key = list(element[key].keys())[0]
        instance_element[key] = element[key][sub_key]
      else:
        instance_element[key] = element[key]
  return instance_element


In [ ]:

parsed_data = [get_instance(element) for element in data_json]
raw_pandas_data = pd.DataFrame(parsed_data)
raw_pandas_data.head()

,codigo,tipo_propiedad,tipo_operacion,precio_venta,area,habitaciones,banos,administracion,parqueaderos,sector,estrato,antiguedad,estado,longitud,latitud,datetime
0,374-M2718950,APARTAMENTO,VENTA Y ARRIENDO,2500000000,320.0,4.0,5.0,2500000,3.0,COUNTRY,6.0,MAS DE 20 ANOS,USADO,-74.026855,4.715475,{'$date': '2023-06-19T05:43:49.763Z'}
1,2510-M4156145,APARTAMENTO,VENTA,485000000,84.0,3.0,3.0,680000,1.0,SANTA BARBARA,6.0,MAS DE 20 ANOS,USADO,-74.0459,4.6945,{'$date': '2023-06-19T05:43:53.411Z'}
2,10312-M2873283,APARTAMENTO,VENTA Y ARRIENDO,1980000000,248.0,3.0,3.0,2200000,3.0,SANTA BARBARA,6.0,MAS DE 20 ANOS,USADO,-74.027916,4.690854,{'$date': '2023-06-19T05:43:58.765Z'}
3,10139-M4343016,APARTAMENTO,VENTA,495000000,77.2,3.0,2.0,367000,1.0,COLINA Y ALREDEDORES,4.0,ENTRE 5 Y 10 ANOS,USADO,-74.0564,4.742382,{'$date': '2023-06-19T05:44:02.598Z'}
4,10139-M3692828,APARTAMENTO,VENTA,1570000000,213.0,3.0,4.0,994000,3.0,COUNTRY,3.0,ENTRE 5 Y 10 ANOS,USADO,-74.028244,4.708852,{'$date': '2023-06-19T05:44:06.286Z'}


In [ ]:
#Printing data base size
raw_pandas_data.shape

(68973, 16)

In [ ]:
#gmaps = googlemaps.Client(key='AIzaSyBoyvv6tLOs7LIs')
reverse_geocode_result = gmaps.reverse_geocode((raw_pandas_data["latitud"][0], raw_pandas_data['longitud'][0]))
print(reverse_geocode_result[0].keys())
reverse_geocode_result[0]["address_components"][-1]["long_name"]



dict_keys(['address_components', 'formatted_address', 'geometry', 'navigation_points', 'place_id', 'types'])


'110121'

In [ ]:
raw_pandas_data_copy = raw_pandas_data.copy()
gmaps_reverse_list = [(raw_pandas_data["latitud"][index], raw_pandas_data["longitud"][index]) for index, instance in raw_pandas_data.iterrows()]




In [ ]:
len(gmaps_reverse_list)

68973

In [ ]:
for gmaps_reverse in gmaps_reverse_list:
  if type(gmaps_reverse[0]) == float:
    reverse_geocode_result = gmaps.reverse_geocode(gmaps_reverse)
  else:
    reverse_geocode_result = None


HTTPError: HTTP Error: 400

In [ ]:
raw_pandas_data_copy["post_code"] = pd.DataFrame(reverse_geocode_result, columns=["post_code"])

In [ ]:
raw_pandas_data_copy.head()

,codigo,tipo_propiedad,tipo_operacion,precio_venta,area,habitaciones,banos,administracion,parqueaderos,sector,estrato,antiguedad,estado,longitud,latitud,datetime,post_code
0,374-M2718950,APARTAMENTO,VENTA Y ARRIENDO,2500000000,320.0,4.0,5.0,2500000,3.0,COUNTRY,6.0,MAS DE 20 ANOS,USADO,-74.026855,4.715475,{'$date': '2023-06-19T05:43:49.763Z'},NaN
1,2510-M4156145,APARTAMENTO,VENTA,485000000,84.0,3.0,3.0,680000,1.0,SANTA BARBARA,6.0,MAS DE 20 ANOS,USADO,-74.0459,4.6945,{'$date': '2023-06-19T05:43:53.411Z'},NaN
2,10312-M2873283,APARTAMENTO,VENTA Y ARRIENDO,1980000000,248.0,3.0,3.0,2200000,3.0,SANTA BARBARA,6.0,MAS DE 20 ANOS,USADO,-74.027916,4.690854,{'$date': '2023-06-19T05:43:58.765Z'},NaN
3,10139-M4343016,APARTAMENTO,VENTA,495000000,77.2,3.0,2.0,367000,1.0,COLINA Y ALREDEDORES,4.0,ENTRE 5 Y 10 ANOS,USADO,-74.0564,4.742382,{'$date': '2023-06-19T05:44:02.598Z'},NaN
4,10139-M3692828,APARTAMENTO,VENTA,1570000000,213.0,3.0,4.0,994000,3.0,COUNTRY,3.0,ENTRE 5 Y 10 ANOS,USADO,-74.028244,4.708852,{'$date': '2023-06-19T05:44:06.286Z'},NaN


In [ ]:
#saving raw data base in csv format
raw_pandas_data.to_csv('builker_raw_data.csv', index=False)